In [ ]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time 
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

: 

In [ ]:
start_month = 2
end_month = 1
fecha = '20230701%'

In [ ]:
query = """
SELECT 
  dp.card_id,
  margin_mtd AS margin,
  d.year_month_char AS month,
  MAX(dim_date) AS maxx
FROM dpr_cross_business.fact_cross_business_insights    cbi
INNER JOIN dpr_shared.dim_date                          d   ON cbi.dim_date = d.date_id
INNER JOIN dpr_shared.dim_stock_unit                    su  ON cbi.dim_stock_unit = su.stock_unit_id
INNER JOIN dpr_shared.dim_product                       dp  ON su.card_id = dp.card_id

WHERE margin_mtd <> 0
  AND d.full_date >= DATE_TRUNC('month', date(getdate()) - interval '{start_month} month')
  AND d.full_date < DATE_TRUNC('month', date(getdate()) - interval '{end_month} month')
GROUP BY 1,2,3
""".format(start_month=start_month, end_month=end_month)
df_margen = run_read_dwd_query(query)
df_margen[["margin"]] = df_margen[["margin"]].astype(float)

In [ ]:
query="""
SELECT DISTINCT 
split_part(bs.upc, '-',1) as region_code,
bs.sku_id,
split_part(bs.upc, ':',4) as IdProd,
bs.addl_product_id as card_id,
bs.upc,
bcat2.name as categoria,
CASE
	WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
	WHEN bcat.name ILIKE 'Res%' THEN 'Res'
	WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
	ELSE bcat.name
END AS subcategoria,
bs2.name as card_name,
CASE
    WHEN ((bs.active_end_date IS NULL OR bs.active_end_date > CURRENT_TIMESTAMP) AND (bs2.active_end_date IS NULL OR bs2.active_end_date > CURRENT_TIMESTAMP)) THEN 'on'
    ELSE 'off'
END as status_frida

FROM postgres_broadleaf_federate."broadleaf.blc_sku" bs
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_product" bp  on bp.product_id = bs.addl_product_id --Conectar sku con la tarjeta
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_sku" bs2 on bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref" bcx on bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and default_reference = 'true' --Relaciones categorias 
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category" bcat on bp.default_category_id = bcat.category_id --Nombre subcategoria
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category" bcat2 on bcx.category_id = bcat2.category_id --Nombre categoria

WHERE bs.archived = 'N'
 AND bs.sndbx_id is null
 AND bp.archived = 'N'
 AND bs.catalog_disc < 0
 AND ((bp.archived = 'N' and bp.sndbx_id is null) or (bp.archived is null))
"""
frida_products = read_connection_data_warehouse.runQuery(query)

In [ ]:
query = """
WITH exploded_data AS (
SELECT DISTINCT
    bs.site_identifier_value AS region_code,
    bo.offer_id,
    MAX(bcs.customer_segment_id) AS customer_segment_id_test,
    --bcs.customer_segment_name,
    MAX(bcs2.customer_segment_id) AS customer_segment_id_control,
    --bcs2.customer_segment_name,
    bo.offer_name AS discount,
    DATE(bo.start_date) AS start_date,
    DATE(bo.end_date) AS end_date,
    bo.offer_value::float as Value_Discount,
    replace(
        substring(
          boic.order_item_match_rule, 
          position('["' in boic.order_item_match_rule) + 2, 
          position('"]' in boic.order_item_match_rule) - position('["' in boic.order_item_match_rule) - 2
    ), '"', '') AS values_string
    
FROM postgres_broadleaf_federate."broadleaf.blc_offer"                          bo
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref"   bocsx   ON (bo.offer_id = bocsx.offer_id AND ((bocsx.archived is NULL OR bocsx.archived='N')))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref"      btcofx  ON (btcofx.offer_id = bo.offer_id)
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria"      boic    ON (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id and (boic.archived is null or boic.archived ='N'))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     bs      ON bs.site_id = bo.catalog_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs     ON bocsx.customer_segment_id = bcs.customer_segment_id AND bcs.customer_segment_name ILIKE '%piloto%'
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs2    ON REPLACE(bcs.customer_segment_name, 'piloto', 'control') = bcs2.customer_segment_name 

WHERE offer_name ILIKE '%\_piloto\_NB\_%'
    AND offer_name ILIKE '{fecha}'
    AND end_date >= DATE_TRUNC('month', date(getdate()) - interval '{start_month} month')
    --AND end_date < DATE_TRUNC('month', date(getdate()) - interval '{end_month} month')
    AND bo.offer_discount_type = 'PERCENT_OFF'
    AND bo.offer_type = 'ORDER_ITEM'
    AND bo.sndbx_id is NULL
    AND (bo.archived is NULL OR bo.archived='N')
    AND (bo.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
    AND bo.automatically_added = 'true'
    --AND bs.site_identifier_value = 'MDE'
GROUP BY 1,2,5,6,7,8,9
), 
 
number_series AS (
  SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10
  UNION ALL SELECT 11 UNION ALL SELECT 12 UNION ALL SELECT 13 UNION ALL SELECT 14 UNION ALL SELECT 15 UNION ALL SELECT 16 UNION ALL SELECT 17 UNION ALL SELECT 18 UNION ALL SELECT 19 UNION ALL SELECT 20
  UNION ALL SELECT 21 UNION ALL SELECT 22 UNION ALL SELECT 23 UNION ALL SELECT 24 UNION ALL SELECT 25 UNION ALL SELECT 26 UNION ALL SELECT 27 UNION ALL SELECT 28 UNION ALL SELECT 29 UNION ALL SELECT 30
  UNION ALL SELECT 31 UNION ALL SELECT 32 UNION ALL SELECT 33 UNION ALL SELECT 34 UNION ALL SELECT 35 UNION ALL SELECT 36 UNION ALL SELECT 37 UNION ALL SELECT 38 UNION ALL SELECT 39 UNION ALL SELECT 40
  UNION ALL SELECT 41 UNION ALL SELECT 42 UNION ALL SELECT 43 UNION ALL SELECT 44 UNION ALL SELECT 45 UNION ALL SELECT 46 UNION ALL SELECT 47 UNION ALL SELECT 48 UNION ALL SELECT 49 UNION ALL SELECT 50
  UNION ALL SELECT 51 UNION ALL SELECT 52 UNION ALL SELECT 53 UNION ALL SELECT 54 UNION ALL SELECT 55 UNION ALL SELECT 56 UNION ALL SELECT 57 UNION ALL SELECT 58 UNION ALL SELECT 59 UNION ALL SELECT 60
  UNION ALL SELECT 61 UNION ALL SELECT 62 UNION ALL SELECT 63 UNION ALL SELECT 64 UNION ALL SELECT 65 UNION ALL SELECT 66 UNION ALL SELECT 67 UNION ALL SELECT 68 UNION ALL SELECT 69 UNION ALL SELECT 70
  UNION ALL SELECT 71 UNION ALL SELECT 72 UNION ALL SELECT 73 UNION ALL SELECT 74 UNION ALL SELECT 75 UNION ALL SELECT 76 UNION ALL SELECT 77 UNION ALL SELECT 78 UNION ALL SELECT 79 UNION ALL SELECT 80
  UNION ALL SELECT 81 UNION ALL SELECT 82 UNION ALL SELECT 83 UNION ALL SELECT 84 UNION ALL SELECT 85 UNION ALL SELECT 86 UNION ALL SELECT 87 UNION ALL SELECT 88 UNION ALL SELECT 89 UNION ALL SELECT 90
  UNION ALL SELECT 91 UNION ALL SELECT 92 UNION ALL SELECT 93 UNION ALL SELECT 94 UNION ALL SELECT 95 UNION ALL SELECT 96 UNION ALL SELECT 97 UNION ALL SELECT 98 UNION ALL SELECT 99 UNION ALL SELECT 100
)

SELECT DISTINCT
    ed.region_code,
    ed.offer_id,
    ed.customer_segment_id_test,
    ed.customer_segment_id_control,
    ed.discount,
    ed.start_date,
    ed.end_date,
    ed.value_discount,
    bs.addl_product_id AS card_id,
    bs2.name,
    bcat2.name AS cat,
    bcat.name as subcat
    
FROM exploded_data                                                                  ed
JOIN number_series                                                                  ns      ON ns.n <= regexp_count(ed.values_string, ',') + 1 AND trim(split_part(ed.values_string, ',', ns.n)) <> ''
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = trim(split_part(ed.values_string, ',', ns.n))::int
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"                bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat2   ON bcx.category_id = bcat2.category_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref"   bcs     ON bo.customer_segment_id = bcs.customer_segment_id

WHERE
    -- FIX SUPER DESCUENTOS
    bcat2.category_id NOT IN ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
""".format(start_month=start_month, end_month=end_month, fecha=fecha)#.format(clientes=tuple(set(clear_buyers[clear_buyers.registered_city == city].customer_id.unique())), city = city)

df_offers = read_connection_data_warehouse.runQuery(query)
df_offers['start_date'] = pd.to_datetime(df_offers['start_date'])
df_offers['end_date'] = pd.to_datetime(df_offers['end_date'])

In [ ]:
df_offers.loc[df_offers.start_date == df_offers.start_date.max()].region_code.value_counts()

SPO    1966
BOG    1636
CMX    1400
BHZ     870
VCP     830
BAQ     682
CWB     596
MDE     445
Name: region_code, dtype: int64

In [ ]:
df_offers_grouped = df_offers.groupby(by = ['region_code', 'offer_id', 'customer_segment_id_test',
       'customer_segment_id_control', 'discount', 'start_date', 'end_date',
       'value_discount']).agg({'card_id': lambda x: x.unique().tolist(), 'name': lambda x: x.unique().tolist()}).reset_index()

In [ ]:
def customers_offer(grupo):
    query = """
    SELECT DISTINCT 
        bcs.customer_segment_id, 
        bcs.external_identifier::int AS customer_id
    FROM postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref" bcs
    WHERE bcs.customer_segment_id = {clientes}
    """.format(clientes=grupo)

    df_clients = read_connection_data_warehouse.runQuery(query)
    return df_clients.customer_id.unique()

In [ ]:
query = """
SELECT 
    s.identifier_value AS city,
    dc.source_id AS customer_id,
    DATE(fs.order_submitted_date) AS submit_date,
    fs.order_id,
    dp.card_id AS padre_sku_id,
    cat.parent_description AS cat,
    CASE
        WHEN cat.description ILIKE 'Pollo%' THEN 'Pollo'
        WHEN cat.description ILIKE 'Res%' THEN 'Res'
        WHEN cat.description ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE cat.description
    END AS subcat,
    dp.card_description AS name,
    CASE
        WHEN s.identifier_value = 'CMX' THEN fs.gmv_pxq_local/19.65
        WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fs.gmv_pxq_local/4.75
        WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fs.gmv_pxq_local/3776
    ELSE fs.gmv_pxq_local 
    END AS gmv_usd,
    CASE
        WHEN s.identifier_value = 'CMX' THEN fsd.product_discount/19.65
        WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fsd.product_discount/4.75
        WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fsd.product_discount/3776
    ELSE fsd.product_discount 
    END AS discount_usd,
    fs.product_quantity_x_step_unit AS cant

FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.order_submitted_date >= DATE_TRUNC('month', date(getdate()) - interval '{start_month} month')
    AND fs.order_submitted_date < DATE_TRUNC('month', date(getdate()) - interval '{end_month} month')
""".format(start_month=start_month, end_month=end_month)

df_sells = run_read_dwd_query(query)
df_sells[["gmv_usd", "cant","discount_usd"]] = df_sells[["gmv_usd", "cant","discount_usd"]].astype(float)
df_sells['submit_date'] = pd.to_datetime(df_sells['submit_date']) 

In [ ]:
def df_info_filter_offer(df_offers_grouped, i, test, control):
    df_filter_test = df_sells[(df_sells['city'] == df_offers_grouped.loc[i,'region_code']) & 
                         (df_sells['customer_id'].isin(test)) & 
                         (df_sells['submit_date'] >= df_offers_grouped.loc[i,'start_date']) & 
                         (df_sells['submit_date'] < df_offers_grouped.loc[i,'end_date']) & 
                         (df_sells['padre_sku_id'].isin(df_offers_grouped.loc[i,'card_id']))].copy()
    df_filter_control = df_sells[(df_sells['city'] == df_offers_grouped.loc[i,'region_code']) & 
                         (df_sells['customer_id'].isin(control)) & 
                         (df_sells['submit_date'] >= df_offers_grouped.loc[i,'start_date']) & 
                         (df_sells['submit_date'] < df_offers_grouped.loc[i,'end_date']) & 
                         (df_sells['padre_sku_id'].isin(df_offers_grouped.loc[i,'card_id']))].copy()
        
    df_filter_test['group'] = 'test'
    df_filter_control['group'] = 'control'
    df_filter_test['all_customer'] = len(test)
    df_filter_control['all_customer'] = len(control)
    df_filter = pd.concat([df_filter_test, df_filter_control]).reset_index(drop=True)
    df_filter['offer_id'] = df_offers_grouped.loc[i,'offer_id']
    df_filter['margin'] = df_filter.padre_sku_id.apply(lambda x: 0.1 if len(df_margen.loc[df_margen.card_id == x]['margin'].unique()) == 0 else df_margen.loc[df_margen.card_id == x]['margin'].unique()[0])
    df_filter['cash_margin'] = df_filter.gmv_usd * df_filter.margin
    df_filter['net_cash_margin'] = df_filter.cash_margin - df_filter.discount_usd
    
    return df_filter

In [ ]:
print(len(df_offers_grouped))

1815


In [ ]:
results_df = pd.DataFrame()

for i in range(len(df_offers_grouped)):
    test = customers_offer(df_offers_grouped.loc[i,'customer_segment_id_test'])
    control = customers_offer(df_offers_grouped.loc[i,'customer_segment_id_control'])
    print(i, df_offers_grouped.loc[i,'offer_id'])
    
    info_offer = df_info_filter_offer(df_offers_grouped, i, test, control)

    results_offer = (info_offer.groupby(by=['city', 'offer_id', 'subcat', 'group', 'all_customer'])
                            .agg(customer_count=('customer_id', 'nunique'),
                                gmv_usd=('gmv_usd', np.sum),
                                discount_usd=('discount_usd', np.sum),
                                quantity=('cant', np.sum),
                                margin=('margin', np.mean),
                                cash_margin=('cash_margin', np.sum),
                                net_cash_margin=('net_cash_margin', np.sum))
                            .reset_index()
                    )
    
    results_offer['convertion_rate'] = results_offer.customer_count / results_offer.all_customer       
    results_offer['AOV'] = results_offer.gmv_usd / results_offer.customer_count
    results_offer['AOV_net'] = results_offer.cash_margin / results_offer.customer_count
    results_offer['dct_avg'] = results_offer.discount_usd / results_offer.customer_count
     
    if len(results_offer) == 2:
        results_df = pd.concat([results_df, results_offer]).reset_index(drop=True)
    elif len(results_offer) == 1 and 'piloto' in results_offer.group:
        zero_row = {
            'city': results_offer.loc[0,'city'],
            'offer_id': results_offer.loc[0,'offer_id'],
            'subcat': results_offer.loc[0,'subcat'],
            'group': 'control',
            'all_customer': len(control),
            'customer_count': 0,
            'gmv_usd': 0,
            'discount_usd': 0,
            'quantity': 0,
            'margin': 0,
            'cash_margin': 0,
            'convertion_rate': 0,
            'AOV': 0,
            'AOV_net': 0,
            'dct_avg': 0
        }
        results_offer = pd.concat([results_offer, zero_row]).reset_index(drop=True)
        results_df = pd.concat([results_df, results_offer]).reset_index(drop=True)
    elif len(results_offer) == 1 and 'control' in results_offer.group:
        zero_row = {
            'city': results_offer.loc[0,'city'],
            'offer_id': results_offer.loc[0,'offer_id'],
            'subcat': results_offer.loc[0,'subcat'],
            'group': 'piloto',
            'all_customer': len(test),
            'customer_count': 0,
            'gmv_usd': 0,
            'discount_usd': 0,
            'quantity': 0,
            'margin': 0,
            'cash_margin': 0,
            'convertion_rate': 0,
            'AOV': 0,
            'AOV_net': 0,
            'dct_avg': 0
        }
        results_offer = pd.concat([results_offer, zero_row]).reset_index(drop=True)
        results_df = pd.concat([results_df, results_offer]).reset_index(drop=True)
    else:
        pass

0 -14520562
1 -14520561
2 -14520560
3 -14520559
4 -14520558
5 -14520557
6 -14520556
7 -14520555
8 -14520554
9 -14520553
10 -14520552
11 -14520551
12 -14520550
13 -14520549
14 -14520548
15 -14520547
16 -14520546
17 -14520545
18 -14520544
19 -14520543
20 -14520542
21 -14520541
22 -14520540
23 -14520539
24 -14520538
25 -14520537
26 -14520536
27 -14520534
28 -14520533
29 -14520532
30 -14520531
31 -14520530
32 -14520529
33 -14520528
34 -14520499
35 -14520498
36 -14520497
37 -14520496
38 -14520495
39 -14520494
40 -14520493
41 -14520491
42 -14520490
43 -14520489
44 -14520488
45 -14520487
46 -14520486
47 -14520485
48 -14520484
49 -14520483
50 -14520482
51 -14520481
52 -14520480
53 -14520479
54 -14520478
55 -14520477
56 -14520476
57 -14520475
58 -14520474
59 -14520473
60 -14520472
61 -14520471
62 -14520439
63 -14520438
64 -14520437
65 -14520436
66 -14520435
67 -14520434
68 -14520433
69 -14520432
70 -14520431
71 -14520430
72 -14520429
73 -14520428
74 -14520427
75 -14520426
76 -14520425
77 -14520

InternalError_: Remote cluster is initializing. Please try again later.


In [ ]:
offers_metrics_df =  pd.merge(df_offers_grouped[['region_code', 'offer_id','discount','value_discount']], 
                              results_df[['offer_id', 'subcat', 'group', 'all_customer', 'customer_count','gmv_usd', 'discount_usd', 'quantity', 'margin', 'cash_margin','net_cash_margin','convertion_rate', 'AOV', 'AOV_net', 'dct_avg']], 
                              left_on='offer_id', right_on='offer_id', how='inner')

In [ ]:
def calculate_ROI(df):
    df.columns = df.iloc[5].unique()
    
    GMV_t = df['test'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['test'].loc['AOV_net']
    dct_t = df['test'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['test'].loc['dct_avg']

    GMV_c = df['control'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['control'].loc['AOV_net']
    dct_c = df['control'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['control'].loc['dct_avg']
    
    net_GMV_t = GMV_t - dct_t
    net_GMV_c = GMV_c - dct_c
    
    diff_net_GMV = net_GMV_t - net_GMV_c
    diff_net_dct = dct_t - dct_c
    
    ROI = diff_net_GMV / diff_net_dct if diff_net_dct != 0 else 0
    
    
    if df['control'].loc['dct_avg'] > df['test'].loc['dct_avg']:
        ROI = 'NO APLICA'

    return diff_net_GMV, diff_net_dct, ROI

In [ ]:
for i,offer in enumerate(offers_metrics_df.offer_id.unique()):
        if offer not in [-14480065]:
                print(i,offer)
                diff_net_GMV, diff_net_dct, ROI = calculate_ROI(offers_metrics_df.loc[offers_metrics_df.offer_id == offer].T)
                offers_metrics_df.loc[offers_metrics_df.offer_id == offer,'diff_net_GMV'] = diff_net_GMV
                offers_metrics_df.loc[offers_metrics_df.offer_id == offer,'diff_net_dct'] = diff_net_dct
                offers_metrics_df.loc[offers_metrics_df.offer_id == offer,'ROI'] = ROI

0 312955
1 312956
2 312957
3 312958
4 312963
5 312964
6 312965
7 312966
8 312968
9 312969
10 312973
11 312975
12 312977
13 312978
14 312979
15 312980
16 312985
17 312986
18 312987
19 312988
20 312992
21 312995
22 312998
23 312999
24 313001
25 313003
26 313006
27 313010
28 313070
29 313071
30 313073
31 313076
32 313078
33 313082
34 313084
35 313085
36 313090
37 313091
38 313092
39 313094
40 313098
41 313099
42 313100
43 313101
44 313106
45 313107
46 313108
47 313110
48 313114
49 313115
50 313116
51 313117
52 313122
53 313123
54 313124
55 313125
56 313130
57 313133
58 313192
59 313193
60 313194
61 313195
62 313200
63 313201
64 313202
65 313204
66 313208
67 313209
68 313210
69 313212
70 313216
71 313217
72 313218
73 313222
74 313223
75 313225
76 313227
77 313230
78 313231
79 313233
80 313234
81 313236
82 313237
83 313243
84 313244
85 313245
86 313248
87 313251
88 313253
89 313318
90 313320
91 313321
92 313323
93 313326
94 313329
95 313330
96 313331
97 313334
98 313336
99 313338
100 313340

In [ ]:
df_offers_grouped.to_csv(f'SD_info offers_{start_month}_{end_month}.csv', index=False)
offers_metrics_df.to_csv(f'RESULTS_SD_ROI_{start_month}_{end_month}.csv', index=False)